In [ ]:
Given the timing, this set will be validated first.

In [4]:
import os
import json
import glob
from lxml import etree

In [5]:
# grab the previous xml processing
_doc_dir = '/Users/sparky/Documents/solr_responses/solr_20150922_docs/'
_xpaths = [
    ['//*', '@schemaLocation'],
    ['//*', '@noNamespaceSchemaLocation']
]

def generate_localname_xpath(tags):
    unchangeds = ['*', '..', '.', '//*']
    return '/'.join(
        ['%s*[local-name()="%s"]' % ('@' if '@' in t else '', t.replace('@', ''))
         if t not in unchangeds else t for t in tags])


def extract_attribs(elem, tags):
    e = extract_elems(elem, tags)
    return list([' '.join(m.strip().split()) for m in e] if isinstance(e, list) else [' '.join(e.split())])


def extract_elems(elem, tags):
    xp = generate_localname_xpath(tags)
    return elem.xpath(xp)


def _clean_content(response):
    response = response.replace('\\\n', '').replace('\r\n', '').replace('\\r', '').replace('\\n', '').replace('\n', '')
    response = response.replace('\\\t', '').replace('\\t', '').replace('\t', '')
    # this is likely useless (mostly issues in the json)
    response = response.replace('\\\\ufffd', '').replace('\\\ufffd', '').replace('\\ufffd', '').replace('\ufffd', '')
    response = response.decode('utf-8', errors='replace').encode('unicode_escape') 
    return response


def _parse_content(response):
    parser = etree.XMLParser()
    return etree.fromstring(response, parser=parser)


def prep_content(filename):
    with open(filename, 'r') as f:
        data = json.loads(f.read())
    response = data.get('raw_content', '')
    response = _clean_content(response)
    return _parse_content(response)

In [6]:
# load the set of identified federal schemas
with open('outputs/federal_schemas_sorted.txt', 'r') as f:
    federal_schemas = f.readlines()

federal_schemas = [f.strip() for f in federal_schemas if f]

In [55]:
# run through our responses
files = glob.glob(_doc_dir + '*.json')
contains_federals = []
for f in files[700000:750000]:
    try:
        xml = prep_content(f)
    except Exception as ex:
        continue

    schemas = []
    for xp in _xpaths:
        schemas += extract_attribs(xml, xp)

    if not schemas:
        continue

    schemas = '  '.join([a.strip() for s in schemas for a in s.split()])

    # and check if there's an intersect in the lists
    if any(f in schemas for f in federal_schemas):
        contains_federals.append(f)

In [56]:
len(contains_federals)

0

In [54]:
with open('outputs/responses_with_federal_schemas.txt', 'a') as g:
    g.write('\n')
    g.write('\n'.join(contains_federals))